# Finetune a Qwen3 model with 700 million parameters

We use a self-curated dataset with [arXiv](https://arxiv.org/) as a basis. The whole dataset is available from [Kaggle](https://www.kaggle.com/datasets/Cornell-University/arxiv). The dataset was filtered to have either `LLM` (only as a whole word), `Large Language Model`, `GRPO` or `GSPO` in the title of the abstract. 

We use the following system prompt:
```
You are an educated researcher and always answer in correct scientific terms.
You are very deep into LLMs and its methodologies. You are very creative.
```

The model should learn how to write abstracts about a certain topic. An example user prompt would be:
```
Write an abstract with the title 'Few-shot training LLMs for project-specific code-summarization'
```

The example abstract is taken from arXiv, so the answer will be:
```
Very large language models (LLMs), such as GPT-3 and Codex have achieved
state-of-the-art performance on several natural-language tasks, and show great
promise also for code. A particularly exciting aspect of LLMs is their knack
for few-shot and zero-shot learning: they can learn to perform a task with very
few examples. Few-shotting has particular synergies in software engineering,
where there are a lot of phenomena (identifier names, APIs, terminology, coding
patterns) that are known to be highly project-specific. However,
project-specific data can be quite limited, especially early in the history of
a project; thus the few-shot learning capacity of LLMs might be very relevant.
In this paper, we investigate the use few-shot training with the very large GPT
(Generative Pre-trained Transformer) Codex model, and find evidence suggesting
that one can significantly surpass state-of-the-art models for
code-summarization, leveraging project-specific training.
```

If finetuning works, we will observe that the models know about `GRPO` which
was introduced *after* the knowledge cutoff. Moreover, the models should adhere
to the format and scientific language of the abstract.

Implementation ideas and parts of the script from https://www.philschmid.de/fine-tune-llms-in-2025

In [ ]:
import torch
# from transformers we just need the AutoTokenizer, the kernel comes from liger
from transformers import AutoTokenizer
# trl is the training framework from Hugging Face
from trl import SFTTrainer, ModelConfig, SFTConfig
# trl uses a dataset for training. Hugging Face provides theses datasets, but we have our own
from datasets import load_dataset
# much faster kernel
from liger_kernel.transformers import AutoLigerKernelForCausalLM

# Arguments and Configuration

In [ ]:
model_name = 'Qwen/Qwen3-0.6B'
dataset_name = 'llm-abstract-dataset.jsonl.xz'
output_dir = "runs/" + model_name.split("/")[-1] + "-" + dataset_name.split(".")[0]

In [ ]:
model_args = ModelConfig(model_name_or_path=model_name, 
                         model_revision='main', 
                         torch_dtype='bfloat16', 
                         trust_remote_code=False, 
                         attn_implementation='flash_attention_2', 
                        )

In [ ]:
training_args = SFTConfig(
     output_dir=output_dir,    
     num_train_epochs=4,
     bf16=True,
     packing=True,
     max_length=1024,
     per_device_train_batch_size=8,
     gradient_accumulation_steps=2,
     gradient_checkpointing=True,
     gradient_checkpointing_kwargs = { "use_reentrant": False },
     learning_rate=2.0e-4,
     lr_scheduler_type="constant",
     use_liger_kernel=True,
     warmup_ratio=0.1,
)

# Load dataset

In [ ]:
train_dataset = load_dataset("json", data_files=dataset_name, split="train")

f'Dataset with {len(train_dataset)} samples and the following features: {train_dataset.features}'

In [ ]:
train_dataset[0]

# Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None: 
    tokenizer.pad_token = tokenizer.eos_token

# Model

In [ ]:
# define model kwargs
model_kwargs = dict(
    revision=model_args.model_revision, # What revision from Huggingface to use, defaults to main
    trust_remote_code=model_args.trust_remote_code, # Whether to trust the remote code, this also you to fine-tune custom architectures
    attn_implementation=model_args.attn_implementation, # What attention implementation to use, defaults to flash_attention_2
    dtype=model_args.torch_dtype, # What torch dtype to use, defaults to auto
    use_cache=False if training_args.gradient_checkpointing else True, # Whether
    low_cpu_mem_usage=True,  # Reduces memory usage on CPU for loading the model
    device_map="cuda"
)

In [ ]:
model = AutoLigerKernelForCausalLM.from_pretrained(model_args.model_name_or_path, **model_kwargs)

# Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer
)

## Training loop

In [ ]:
train_result = trainer.train()
# log metrics
metrics = train_result.metrics
metrics['train_samples'] = len(train_dataset)
trainer.save_metrics('train', metrics)
trainer.save_state()
metrics

# Save model

In [ ]:
# Restore k,v cache for fast inference
trainer.model.config.use_cache = True
trainer.save_model(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)
f"saved to {training_args.output_dir}"